In [94]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets
import plotly.express as px

from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_recall_curve, average_precision_score, \
    roc_auc_score, roc_curve, auc, recall_score, precision_score
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

import seaborn as sns
import scipy.stats
from scipy import stats
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from xgboost import XGBClassifier

import time

pd.set_option("display.max_columns", None)

database = pd.read_csv('../data/weatherAUS.csv')

In [2]:
targetNulls = widgets.ToggleButtons(
    options=['No', 'Yes', 'Delete'],
    description='Optios:',
    disabled=False,
    tooltips=['No', 'Yes', 'Delete']
)
print("Tractament del target (nulls)")
targetNulls

Tractament del target (nulls)


ToggleButtons(description='Optios:', options=('No', 'Yes', 'Delete'), tooltips=('No', 'Yes', 'Delete'), value=…

In [3]:
balance = widgets.ToggleButtons(
    options=['Oversampling', 'Oversampling Smote' ,'Subsampling','None'],
    description='Optios:',
    disabled=False,
    tooltips=['Oversampling', 'Oversampling Smote' ,'Subsampling', 'None']
)
print("Balanceig de les dades")
balance

Balanceig de les dades


ToggleButtons(description='Optios:', options=('Oversampling', 'Oversampling Smote', 'Subsampling', 'None'), to…

In [4]:
atributeNulls = widgets.ToggleButtons(
    options=['Mode', 'Median', 'Mean'],
    description='Optios:',
    disabled=False,
    tooltips=['Mode', 'Median', 'Mean']
)
print("Tractament de les dades continues")
display(atributeNulls)

Tractament de les dades continues


ToggleButtons(description='Optios:', options=('Mode', 'Median', 'Mean'), tooltips=('Mode', 'Median', 'Mean'), …

In [95]:
if(targetNulls.value == 'No'):
    database['RainTomorrow'] = [1 if i=='Yes' else 0 for i in database['RainTomorrow'] ]
elif(targetNulls.value == 'Yes'):
    database['RainTomorrow']=[0 if i=='No' else 1 for i in database['RainTomorrow'] ]
else:
    database = database.dropna(axis=0, subset=['RainTomorrow'])
    
cols_to_drop=['Date']
database.drop(columns=cols_to_drop,inplace=True)

count_class_0, count_class_1 = database.RainTomorrow.value_counts()
print(count_class_0)
if(balance.value == 'Oversampling'):
    df_class_0 = database[database['RainTomorrow'] == 0]
    df_class_1 = database[database['RainTomorrow'] == 1]
    
    df_class_1_over = df_class_1.sample(count_class_0, replace=True)
    database = pd.concat([df_class_0, df_class_1_over], axis=0)
    
elif(balance.value == 'Subsampling'):
    df_class_0 = database[database['RainTomorrow'] == 0]
    df_class_1 = database[database['RainTomorrow'] == 1]

    df_class_0_under = df_class_0.sample(count_class_1)
    database = pd.concat([df_class_0_under, df_class_1], axis=0)

database['RainTomorrow'] = [1 if i=='Yes' else 0 for i in database['RainTomorrow'] ]

x=database.drop(['RainTomorrow'],axis=1)
y=database['RainTomorrow']

categorical_cols = list(x.select_dtypes(include=['object']).columns)
continuous_cols = list(x.select_dtypes(include=['float64']).columns)

for i in categorical_cols:
    x[i].fillna(x[i].mode()[0], inplace=True)

categorical_cols = list(x.select_dtypes(include=['object']).columns)
x.drop(columns=categorical_cols,inplace=True)

if(targetNulls.value == 'Mode'):
    for column in continuous_cols:
        x[column].fillna(x[column].mode()[0], inplace = True)

elif(targetNulls.value == 'Median'):

    for column in continuous_cols:
        x[column].fillna(x[column].median(), inplace = True)
else:
    for column in continuous_cols:
        x[column].fillna(x[column].mean(), inplace = True)

# transformer = ColumnTransformer(transformers=[('ohe', OneHotEncoder(sparse='False',drop='first'), categorical_cols)],remainder='passthrough')
# x=transformer.fit_transform(x)

if balance.value == 'Oversampling Smote':
    smote=SMOTE()
    x,y=smote.fit_resample(x,y)

    print(x.shape,y.shape)
    
# x=pd.DataFrame(x.toarray())

def check_skewness(x):
    
    skew_limit=0.75
    skew_value=x[x.columns].skew()
    skew_cols=skew_value[abs(skew_value)>skew_limit]
    #print(skew_cols)
    cols=skew_cols.index
    return cols
    
skewed_col=check_skewness(x)
pt=PowerTransformer(standardize=False)
x[skewed_col]=pt.fit_transform(x[skewed_col])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(X_train.shape)
sc=StandardScaler(with_mean=False)
sc=sc.fit(X_train)
X_train=sc.transform(X_train)
X_test=sc.transform(X_test)

print("Opcio,s escollides:")
print("--------------------------------------")
print("Target NUllS :", targetNulls.value)
print("Balance :", balance.value)
print("Dades continues Nulls:", atributeNulls.value)



rf=RandomForestClassifier(n_estimators= 50, max_samples= 0.3, max_features='log2', criterion= 'gini', bootstrap= False)
rf.fit(X_train,y_train)
print("Model entrenat")

113583
(181732, 16)
Opcio,s escollides:
--------------------------------------
Target NUllS : No
Balance : Oversampling
Dades continues Nulls: Mode
Model entrenat


### Entra els valors per predir si ploura dema o no

In [9]:
database2 = pd.read_csv('../data/weatherAUS.csv');
database = pd.read_csv('../data/weatherAUS.csv')
database2 = database2.dropna()

In [10]:
locations = database.Location.unique()
loc = widgets.ToggleButtons(
    options=locations,
    description='Options:',
    disabled=False,
)
print("Tractament de les dades continues")
display(loc)

Tractament de les dades continues


ToggleButtons(description='Options:', options=('Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree', 'N…

In [11]:
minTemp = widgets.FloatSlider(
    value=0,
    description='Value:',
    min=0,
    step=0.1,
    readout_format='.1f'
)
print("Temperatura minima (celsius):")
display(minTemp)

Temperatura minima (celsius):


FloatSlider(value=0.0, description='Value:', readout_format='.1f')

In [12]:
maxTemp = widgets.FloatSlider(
    value=0,
    description='Value:',
    min=database.MaxTemp.min(),
    step=0.1,
    readout_format='.1f'
)
print("Temperatura maxima (celsius):")
display(maxTemp)

Temperatura maxima (celsius):


FloatSlider(value=0.0, description='Value:', min=-4.8, readout_format='.1f')

In [15]:
raifall = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.Rainfall.min(),
    max = 500,
    step=0.1,
    readout_format='.1f'
)
print("Rainfall (mm):")
display(raifall)

Rainfall (mm):


FloatSlider(value=0.0, description='Value:', max=500.0, readout_format='.1f')

In [50]:
evaporation = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.Evaporation.min(),
    step=0.1,
    readout_format='.1f'
)
print("evaporació (mm):")
display(evaporation)

evaporació (mm):


FloatSlider(value=0.0, description='Value:', readout_format='.1f')

In [98]:
sunshine = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.Sunshine.min(),
    step=0.1,
    readout_format='.1f'
)
print("num d'hores que va fer sol:")
display(sunshine)

num d'hores que va fer sol:


FloatSlider(value=0.0, description='Value:', readout_format='.1f')

In [16]:
Wind = database2.WindGustDir.unique()
wind = widgets.ToggleButtons(
    options=Wind,
    description='Options:',
    disabled=False,
)
print("Tractament de les dades continues")
display(wind)

Tractament de les dades continues


ToggleButtons(description='Options:', options=('SSW', 'S', 'NNE', 'WNW', 'N', 'SE', 'ENE', 'NE', 'E', 'SW', 'W…

In [17]:
windSpeed = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.WindGustSpeed.min(),
    max = 150,
    step=1,
    readout_format='.1f'
)
print("WindGustSpeed (km/h):")
display(windSpeed)

WindGustSpeed (km/h):


FloatSlider(value=6.0, description='Value:', max=150.0, min=6.0, readout_format='.1f', step=1.0)

In [18]:
Wind9 = database2.WindDir9am.unique()
wind9 = widgets.ToggleButtons(
    options=Wind9,
    description='Options:',
    disabled=False,
)
print("Tractament de les dades continues")
display(wind9)

Tractament de les dades continues


ToggleButtons(description='Options:', options=('ENE', 'SSE', 'NNE', 'WNW', 'NW', 'N', 'S', 'SE', 'NE', 'W', 'S…

In [19]:
Wind3 = database2.WindDir3am.unique()
wind3 = widgets.ToggleButtons(
    options=Wind3,
    description='Options:',
    disabled=False,
)
print("Tractament de les dades continues")
display(wind3)

Tractament de les dades continues


ToggleButtons(description='Options:', options=('ENE', 'SSE', 'NNE', 'WNW', 'NW', 'N', 'S', 'SE', 'NE', 'W', 'S…

In [22]:
windSpeed9 = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.WindSpeed9am.min(),
    max = 150,
    step=1,
    readout_format='.1f'
)
print("WindGustSpeed (km/h):")
display(windSpeed9)

WindGustSpeed (km/h):


FloatSlider(value=0.0, description='Value:', max=150.0, readout_format='.1f', step=1.0)

In [24]:
windSpeed3 = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.WindSpeed3pm.min(),
    max = 150,
    step=1,
    readout_format='.1f'
)
print("WindGustSpeed (km/h):")
display(windSpeed3)

WindGustSpeed (km/h):


FloatSlider(value=0.0, description='Value:', max=150.0, readout_format='.1f', step=1.0)

In [52]:
humidity9 = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.Humidity9am.min(),
    max = 150,
    step=1,
    readout_format='.1f'
)
print("humetat 9am (tant per cent):")
display(humidity9)

humetat(tant per cent):


FloatSlider(value=0.0, description='Value:', max=150.0, readout_format='.1f', step=1.0)

In [54]:
humidity3 = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.Humidity3pm.min(),
    max = 150,
    step=1,
    readout_format='.1f'
)
print("humetat 3pm(tant per cent):")
display(humidity3)

humetat 3pm(tant per cent):


FloatSlider(value=0.0, description='Value:', max=150.0, readout_format='.1f', step=1.0)

In [25]:
pressure9am = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.Pressure9am.min(),
    max = database.Pressure9am.max(),
    step=1,
    readout_format='.1f'
)
print("presion atmosfeica a las 9 (hpa):")
display(pressure9am)

presion atmosfeica a las 9 (hpa):


FloatSlider(value=980.5, description='Value:', max=1041.0, min=980.5, readout_format='.1f', step=1.0)

In [55]:
pressure3 = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.Pressure3pm.min(),
    max = database.Pressure3pm.max(),
    step=1,
    readout_format='.1f'
)
print("presion atmosfeica a las 3 (hpa):")
display(pressure3pm)

presion atmosfeica a las 3 (hpa):


FloatSlider(value=980.5, description='Value:', max=1041.0, min=980.5, readout_format='.1f', step=1.0)

In [56]:
cloud9 = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.Cloud9am.min(),
    step=1,
    readout_format='.1f'
)
print("nuvols a les 9am (oktas):")
display(cloud9)

nuvols a les 9am (oktas):


FloatSlider(value=0.0, description='Value:', readout_format='.1f', step=1.0)

In [57]:
cloud3 = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = database.Cloud3pm.min(),
    step=1,
    readout_format='.1f'
)
print("nuvols a les 9am (oktas):")
display(cloud3)

nuvols a les 9am (oktas):


FloatSlider(value=0.0, description='Value:', readout_format='.1f', step=1.0)

In [27]:
temp9 = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = -15,
    step=1,
    readout_format='.1f'
)
print("Temperatura a les 9 (degrees C):")
display(temp9)

Temperatura a les 9 (degrees C):


FloatSlider(value=0.0, description='Value:', min=-15.0, readout_format='.1f', step=1.0)

In [28]:
temp3 = widgets.FloatSlider(
    value = 0,
    description='Value:',
    min = -15,
    step=1,
    readout_format='.1f'
)
print("Temperatura a les 3 (degrees C):")
display(temp3)

Temperatura a les 3 (degrees C):


FloatSlider(value=0.0, description='Value:', min=-15.0, readout_format='.1f', step=1.0)

In [30]:
Raintoday = database2.RainToday.unique()
raintoday = widgets.ToggleButtons(
    options=Raintoday,
    description='Options:',
    disabled=False,
)
print("Va plourè ahir")
display(raintoday)

Va plourè ahir


ToggleButtons(description='Options:', options=('No', 'Yes'), value='No')

In [108]:
newcolumns = transformer.named_transformers_['ohe'].get_feature_names()
values = [
#     loc.value, 
    minTemp.value, 
    maxTemp.value, 
    raifall.value, 
    evaporation.value,
    sunshine.value,
#     wind.value, 
    windSpeed.value, 
#     wind9.value, 
#     wind3.value, 
    windSpeed9.value,
    windSpeed3.value,
    humidity9.value,
    humidity3.value,
    pressure9am.value, 
    pressure3pm.value, 
    cloud9.value,
    cloud3.value,
    temp9.value, 
    temp3.value, 
#     raintoday
]

values = np.array(values).reshape(1,-1)
y_pred = rf.predict(values)
y_pred[0]
if y_pred[0] == 0:
    print("No ploura")
else:
    print("Si ploura")


No ploura
